In [2]:
!pip install neattext

     |████████████████████████████████| 114 kB 5.6 MB/s 


In [72]:
import pandas as pd
import numpy as np
import neattext as nf
import re

In [73]:
ams = pd.read_csv('/content/drive/MyDrive/T5_Camp/animes.csv')

In [74]:
delete_genres = ['hentai', 'harem', 'yaoi', 'Yuri', 'ecchi', 'twincest', 'incest', 'lemon', 'smut']

In [75]:
ams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19311 entries, 0 to 19310
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   uid         19311 non-null  int64  
 1   title       19311 non-null  object 
 2   synopsis    18336 non-null  object 
 3   genre       19311 non-null  object 
 4   aired       19311 non-null  object 
 5   episodes    18605 non-null  float64
 6   members     19311 non-null  int64  
 7   popularity  19311 non-null  int64  
 8   ranked      16099 non-null  float64
 9   score       18732 non-null  float64
 10  img_url     19131 non-null  object 
 11  link        19311 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 1.8+ MB


In [76]:
def clean_animes(animes):
  animes['score'] = animes['score'].astype(str)

  animes['synopsis'].fillna('None', inplace=True)
  animes['genre'].fillna('None', inplace=True)
  animes['episodes'].fillna(0, inplace=True)
  animes['score'].fillna('-', inplace=True)
  animes['img_url'].fillna('https://i.postimg.cc/fL9WXt0H/Capture.png', inplace=True)

  animes['episodes'] = animes['episodes'].astype(int)
  return animes

ams = clean_animes(ams)
ams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19311 entries, 0 to 19310
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   uid         19311 non-null  int64  
 1   title       19311 non-null  object 
 2   synopsis    19311 non-null  object 
 3   genre       19311 non-null  object 
 4   aired       19311 non-null  object 
 5   episodes    19311 non-null  int64  
 6   members     19311 non-null  int64  
 7   popularity  19311 non-null  int64  
 8   ranked      16099 non-null  float64
 9   score       19311 non-null  object 
 10  img_url     19311 non-null  object 
 11  link        19311 non-null  object 
dtypes: float64(1), int64(4), object(7)
memory usage: 1.8+ MB


In [77]:
ams

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...
...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"['Sci-Fi', 'Adventure', 'Comedy', 'Magic']","Oct 6, 2016 to Dec 29, 2016",13,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers
19307,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","['Adventure', 'Fantasy', 'Magic', 'Martial Art...","Apr 6, 1995 to Mar 28, 1996",52,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi
19308,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","['Mystery', 'Horror', 'Supernatural']","Oct 22, 2000 to Mar 25, 2001",19,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...


In [78]:
# delete unnecessary genre
def ignore_genre(genre):
  genre_mask = ams['genre'].str.contains(genre, case=False) == False
  return ams[genre_mask]


for gen in delete_genres:
  ams = ignore_genre(gen)

ams

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...
...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"['Sci-Fi', 'Adventure', 'Comedy', 'Magic']","Oct 6, 2016 to Dec 29, 2016",13,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers
19307,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","['Adventure', 'Fantasy', 'Magic', 'Martial Art...","Apr 6, 1995 to Mar 28, 1996",52,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi
19308,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","['Mystery', 'Horror', 'Supernatural']","Oct 22, 2000 to Mar 25, 2001",19,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...


In [79]:
def extract_years(text):
    years = []

    pattern = re.compile('\d{4}')
    matches = pattern.finditer(text)
    for match in matches:
        years.append(match.group(0))

    return years

def extract_dates(anime_df):
    years = anime_df['aired'].apply(lambda x: extract_years(x))
    anime_df['release_date'] = [e[0] if e else -1 for e in years]

    end_dates = []
    for e in years:
        try:
            end_dates.append(e[1])
        except:
            end_dates.append(-1)

    anime_df['end_date'] = end_dates
    return anime_df


ams = extract_dates(ams)
ams

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,release_date,end_date
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,2015,2016
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,2014,2015
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,2017,2017
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,2009,2010
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,2017,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"['Sci-Fi', 'Adventure', 'Comedy', 'Magic']","Oct 6, 2016 to Dec 29, 2016",13,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers,2016,2016
19307,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","['Adventure', 'Fantasy', 'Magic', 'Martial Art...","Apr 6, 1995 to Mar 28, 1996",52,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi,1995,1996
19308,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","['Mystery', 'Horror', 'Supernatural']","Oct 22, 2000 to Mar 25, 2001",19,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...,2000,2001
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...,2002,-1


In [80]:
ams['release_date'] = ams['release_date'].astype(int)
ams['end_date'] = ams['end_date'].astype(int)
ams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15694 entries, 0 to 19310
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uid           15694 non-null  int64  
 1   title         15694 non-null  object 
 2   synopsis      15694 non-null  object 
 3   genre         15694 non-null  object 
 4   aired         15694 non-null  object 
 5   episodes      15694 non-null  int64  
 6   members       15694 non-null  int64  
 7   popularity    15694 non-null  int64  
 8   ranked        15160 non-null  float64
 9   score         15694 non-null  object 
 10  img_url       15694 non-null  object 
 11  link          15694 non-null  object 
 12  release_date  15694 non-null  int64  
 13  end_date      15694 non-null  int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 1.8+ MB


In [81]:
ams

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,release_date,end_date
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,2015,2016
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,2014,2015
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,2017,2017
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,2009,2010
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,2017,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"['Sci-Fi', 'Adventure', 'Comedy', 'Magic']","Oct 6, 2016 to Dec 29, 2016",13,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers,2016,2016
19307,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","['Adventure', 'Fantasy', 'Magic', 'Martial Art...","Apr 6, 1995 to Mar 28, 1996",52,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi,1995,1996
19308,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","['Mystery', 'Horror', 'Supernatural']","Oct 22, 2000 to Mar 25, 2001",19,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...,2000,2001
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"['Action', 'Adventure', 'Comedy', 'Historical'...","Dec 21, 2002",1,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...,2002,-1


In [82]:
ams['genre'] = ams['genre'].apply(lambda x: nf.remove_special_characters(x).replace(' ', ', '))
ams

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,release_date,end_date
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"Comedy, Sports, Drama, School, Shounen","Oct 4, 2015 to Mar 27, 2016",25,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,2015,2016
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"Drama, Music, Romance, School, Shounen","Oct 10, 2014 to Mar 20, 2015",22,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,2014,2015
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"SciFi, Adventure, Mystery, Drama, Fantasy","Jul 7, 2017 to Sep 29, 2017",13,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,2017,2017
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","Action, Military, Adventure, Comedy, Drama, Ma...","Apr 5, 2009 to Jul 4, 2010",64,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,2009,2010
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"Action, Mystery, Supernatural, Vampire","Jan 6, 2017",1,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,2017,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"SciFi, Adventure, Comedy, Magic","Oct 6, 2016 to Dec 29, 2016",13,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers,2016,2016
19307,123,Fushigi Yuugi,"While visiting the National Library, junior-hi...","Adventure, Fantasy, Magic, Martial, Arts, Come...","Apr 6, 1995 to Mar 28, 1996",52,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi,1995,1996
19308,1281,Gakkou no Kaidan,"Years ago, all of the ghosts in a haunted scho...","Mystery, Horror, Supernatural","Oct 22, 2000 to Mar 25, 2001",19,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...,2000,2001
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"Action, Adventure, Comedy, Historical, Demons,...","Dec 21, 2002",1,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...,2002,-1


In [83]:
syn = ams['synopsis']
syn

0        Following their participation at the Inter-Hig...
1        Music accompanies the path of the human metron...
2        The Abyss—a gaping chasm stretching down into ...
3        "In order for something to be obtained, someth...
4        After helping revive the legendary vampire Kis...
                               ...                        
19306    Cocona is an average middle schooler living wi...
19307    While visiting the National Library, junior-hi...
19308    Years ago, all of the ghosts in a haunted scho...
19309    Inuyasha and company have finally destroyed Na...
19310    The year is Universal Century 0093. Char Aznab...
Name: synopsis, Length: 15694, dtype: object

In [84]:
syn = syn.astype('str')
syn = syn.apply(nf.remove_multiple_spaces)
syn = syn.apply(nf.remove_special_characters)
ams['synopsis'] = syn
ams

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,release_date,end_date
0,28891,Haikyuu!! Second Season,Following their participation at the InterHigh...,"Comedy, Sports, Drama, School, Shounen","Oct 4, 2015 to Mar 27, 2016",25,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,2015,2016
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"Drama, Music, Romance, School, Shounen","Oct 10, 2014 to Mar 20, 2015",22,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,2014,2015
2,34599,Made in Abyss,The Abyssa gaping chasm stretching down into t...,"SciFi, Adventure, Mystery, Drama, Fantasy","Jul 7, 2017 to Sep 29, 2017",13,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,2017,2017
3,5114,Fullmetal Alchemist: Brotherhood,In order for something to be obtained somethin...,"Action, Military, Adventure, Comedy, Drama, Ma...","Apr 5, 2009 to Jul 4, 2010",64,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,2009,2010
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"Action, Mystery, Supernatural, Vampire","Jan 6, 2017",1,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,2017,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"SciFi, Adventure, Comedy, Magic","Oct 6, 2016 to Dec 29, 2016",13,134252,843,1070.0,7.73,https://cdn.myanimelist.net/images/anime/4/822...,https://myanimelist.net/anime/32979/Flip_Flappers,2016,2016
19307,123,Fushigi Yuugi,While visiting the National Library juniorhigh...,"Adventure, Fantasy, Magic, Martial, Arts, Come...","Apr 6, 1995 to Mar 28, 1996",52,84407,1292,1071.0,7.73,https://cdn.myanimelist.net/images/anime/2/201...,https://myanimelist.net/anime/123/Fushigi_Yuugi,1995,1996
19308,1281,Gakkou no Kaidan,Years ago all of the ghosts in a haunted schoo...,"Mystery, Horror, Supernatural","Oct 22, 2000 to Mar 25, 2001",19,83093,1314,1073.0,7.73,https://cdn.myanimelist.net/images/anime/9/183...,https://myanimelist.net/anime/1281/Gakkou_no_K...,2000,2001
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"Action, Adventure, Comedy, Historical, Demons,...","Dec 21, 2002",1,71989,1469,1077.0,7.73,https://cdn.myanimelist.net/images/anime/1162/...,https://myanimelist.net/anime/450/InuYasha_Mov...,2002,-1


In [85]:
ams.drop(columns=['aired', 'members', 'popularity', 'ranked', 'link'], inplace=True)
ams

,uid,title,synopsis,genre,episodes,score,img_url,release_date,end_date
0,28891,Haikyuu!! Second Season,Following their participation at the InterHigh...,"Comedy, Sports, Drama, School, Shounen",25,8.82,https://cdn.myanimelist.net/images/anime/9/766...,2015,2016
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"Drama, Music, Romance, School, Shounen",22,8.83,https://cdn.myanimelist.net/images/anime/3/671...,2014,2015
2,34599,Made in Abyss,The Abyssa gaping chasm stretching down into t...,"SciFi, Adventure, Mystery, Drama, Fantasy",13,8.83,https://cdn.myanimelist.net/images/anime/6/867...,2017,2017
3,5114,Fullmetal Alchemist: Brotherhood,In order for something to be obtained somethin...,"Action, Military, Adventure, Comedy, Drama, Ma...",64,9.23,https://cdn.myanimelist.net/images/anime/1223/...,2009,2010
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"Action, Mystery, Supernatural, Vampire",1,8.83,https://cdn.myanimelist.net/images/anime/3/815...,2017,-1
...,...,...,...,...,...,...,...,...,...
19306,32979,Flip Flappers,Cocona is an average middle schooler living wi...,"SciFi, Adventure, Comedy, Magic",13,7.73,https://cdn.myanimelist.net/images/anime/4/822...,2016,2016
19307,123,Fushigi Yuugi,While visiting the National Library juniorhigh...,"Adventure, Fantasy, Magic, Martial, Arts, Come...",52,7.73,https://cdn.myanimelist.net/images/anime/2/201...,1995,1996
19308,1281,Gakkou no Kaidan,Years ago all of the ghosts in a haunted schoo...,"Mystery, Horror, Supernatural",19,7.73,https://cdn.myanimelist.net/images/anime/9/183...,2000,2001
19309,450,InuYasha Movie 2: Kagami no Naka no Mugenjo,Inuyasha and company have finally destroyed Na...,"Action, Adventure, Comedy, Historical, Demons,...",1,7.73,https://cdn.myanimelist.net/images/anime/1162/...,2002,-1


In [86]:
ams.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15694 entries, 0 to 19310
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   uid           15694 non-null  int64 
 1   title         15694 non-null  object
 2   synopsis      15694 non-null  object
 3   genre         15694 non-null  object
 4   episodes      15694 non-null  int64 
 5   score         15694 non-null  object
 6   img_url       15694 non-null  object
 7   release_date  15694 non-null  int64 
 8   end_date      15694 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.2+ MB


# Save Filtered Animes in CSV File for Deployment

In [87]:
ams.to_csv('/content/drive/MyDrive/T5_Camp/filtered_animes.csv', index=False)

---